In [1]:
from tensorly.tenalg import multi_mode_dot
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from utils import mape, mae, rmse
from data_loaders import load_dataset

/home/uras/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [111]:
DATASET = 'chars' # fun, eps, chars
RANK = 80
LAMBDA = 0.01

def reconstruct(seed):
    f1 = pd.read_csv(f'P-Tucker/results/{DATASET}/{RANK}_{LAMBDA}_{seed}/FACTOR0', header = None, sep='\t').to_numpy()
    f2 = pd.read_csv(f'P-Tucker/results/{DATASET}/{RANK}_{LAMBDA}_{seed}/FACTOR1', header = None, sep='\t').to_numpy()
    f3 = pd.read_csv(f'P-Tucker/results/{DATASET}/{RANK}_{LAMBDA}_{seed}/FACTOR2', header = None, sep='\t').to_numpy()

    df_core = pd.read_csv(f'P-Tucker/results/{DATASET}/{RANK}_{LAMBDA}_{seed}/CORETENSOR', header = None, sep='\t')
    values = df_core[3].to_numpy()
    core = values.reshape(RANK, RANK, RANK)
    X_hat = multi_mode_dot(core, [f1, f2, f3])
    return X_hat

In [112]:
metrics = {'rmse':[], 'mae':[], 'mape':[]}
for seed in [0]:
    X_hat = reconstruct(seed)
    tr_idxs, tr_vals, te_idxs, te_vals, data_shape = load_dataset(DATASET, seed=seed)
    te_idxs = te_idxs.astype(int)

    preds = []
    for index in te_idxs:
        pred = X_hat[index[0], index[1], index[2]]
        preds.append(pred)

    metrics['rmse'].append(rmse(te_vals, preds))
    metrics['mae'].append(mae(te_vals, preds)) 
    metrics['mape'].append(mape(te_vals, preds))

for m, vals in metrics.items():
    print(f'{m}: {np.mean(vals)} +/- {np.std(vals, ddof=1)}')

loading chars dataset..
rmse: 0.26899025419149314 +/- nan
mae: 0.17178863526554747 +/- nan
mape: 72.67205459431894 +/- nan


/home/uras/tmp/ENTER/envs/pyg/lib/python3.10/site-packages/numpy/core/_methods.py:265: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/uras/tmp/ENTER/envs/pyg/lib/python3.10/site-packages/numpy/core/_methods.py:257: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
